In [2]:
import pandas as pd
import numpy as np

In [88]:
combined = pd.concat([pd.read_csv("results/results_0-16.csv"), pd.read_csv("results/results_21.csv")], ignore_index=True)
combined.head()

,name,model,linear_train,linear_test,linear_holdout,rf_train,rf_test,rf_holdout
0,cancer,truth,0.788041,0.695906,0.6,0.93458,0.718129,0.666667
1,cancer,rf,0.788041,0.695906,0.6,0.93458,0.718129,0.666667
2,cancer,pandas,0.788041,0.695906,0.6,0.93458,0.718129,0.666667
3,cancer,tfdv,0.788041,0.695906,0.6,0.93458,0.718129,0.666667
4,cancer,autogluon,0.788041,0.695906,0.6,0.93458,0.718129,0.666667


In [89]:
name_order = combined["name"].unique()
col_order = []
for model in ["logistic regression", "random forest"]:
    for feature_type in ["truth", "pandas", "tfdv", "autogluon", "rf"]:
        col_order.append((model, feature_type))

In [90]:
results = combined[["name", "model", "linear_holdout", "rf_holdout"]].rename({"name": "dataset", "linear_holdout":"logistic regression", "rf_holdout":"random forest"}, axis=1
                                                                  ).pivot(index="dataset", columns="model").loc[name_order, col_order]
results

logistic regression                                          \
model                   truth    pandas      tfdv autogluon        rf   
dataset                                                                 
cancer               0.600000  0.600000  0.600000  0.600000  0.600000   
mfeat                0.962000  0.962000  0.962000  0.962000  0.962000   
nursery              0.920602  0.920833  0.920602  0.920602  0.920602   
audiology            0.734783  0.717391  0.734783  0.700000  0.734783   
hayes                0.755556  0.585185  0.585185  0.585185  0.518519   
supreme              0.994328  0.764488  0.763502  0.764488  0.991862   
flare                0.907692  0.907692  0.907692  0.907692  0.907692   
kropt                0.350677  0.309551  0.309016  0.309016  0.335816   
boxing               0.874074  0.629630  0.659259  0.659259  0.659259   
flags                0.748718  0.625641  0.615385  0.651282  0.620513   
diggle               1.000000  1.000000  1.000000  1.000000  1.000000   
heart                0.842623  0.839344  0.832787  0.839344  0.842623   
sleuth               0.711111  0.722222  0.722222  0.722222  0.711111   
apnea2               0.920000  0.917895  0.920000  0.920000  0.911579   
auto-mpg             0.977215  0.982278  0.830380  0.830380  0.807595   
churn                0.793045  0.788786  0.790774  0.790774  0.790490   
nyc                  0.602915  0.602915  0.602915  0.606025  0.606025   
zoo                  0.711111  0.622222  0.711111  0.688889  0.711111   

          random forest                                          
model             truth    pandas      tfdv autogluon        rf  
dataset                                                          
cancer         0.666667  0.666667  0.666667  0.666667  0.666667  
mfeat          0.941000  0.941000  0.941000  0.941000  0.941000  
nursery        0.982407  0.978781  0.982407  0.982407  0.982407  
audiology      0.721739  0.713043  0.721739  0.708696  0.721739  
hayes          0.785185  0.711111  0.711111  0.711111  0.644444  
supreme        0.994081  0.994081  0.995068  0.994081  0.995561  
flare          0.892308  0.886154  0.901538  0.895385  0.876923  
kropt          0.687847  0.626301  0.653742  0.653742  0.563899  
boxing         0.777778  0.629630  0.666667  0.666667  0.666667  
flags          0.723077  0.769231  0.743590  0.733333  0.728205  
diggle         1.000000  1.000000  1.000000  1.000000  1.000000  
heart          0.862295  0.849180  0.832787  0.849180  0.862295  
sleuth         0.766667  0.766667  0.766667  0.766667  0.766667  
apnea2         0.901053  0.886316  0.892632  0.892632  0.911579  
auto-mpg       0.941772  0.954430  0.762025  0.762025  0.762025  
churn          0.787651  0.779134  0.778992  0.778424  0.778424  
nyc            0.669388  0.669388  0.689602  0.692517  0.692517  
zoo            0.777778  0.644444  0.688889  0.711111  0.711111

In [91]:
def row_diffs(row, classification=True):
    if classification:
        models = ['logistic regression', "random forest"]
    else:
        models = ['linear regression', "random forest"]
    features = ["pandas", "rf", "tfdv", "autogluon"]
    for model in models:
        truth = row[model]["truth"]
        for feature in features:
            row[model][feature] -= truth
    return row
            
    
results_table = (results.apply(row_diffs, axis=1) * 100).round(2)
results_table

logistic regression                                random forest  \
model                   truth pandas   tfdv autogluon     rf         truth   
dataset                                                                      
cancer                  60.00   0.00   0.00      0.00   0.00         66.67   
mfeat                   96.20   0.00   0.00      0.00   0.00         94.10   
nursery                 92.06   0.02   0.00      0.00   0.00         98.24   
audiology               73.48  -1.74   0.00     -3.48   0.00         72.17   
hayes                   75.56 -17.04 -17.04    -17.04 -23.70         78.52   
supreme                 99.43 -22.98 -23.08    -22.98  -0.25         99.41   
flare                   90.77   0.00   0.00      0.00   0.00         89.23   
kropt                   35.07  -4.11  -4.17     -4.17  -1.49         68.78   
boxing                  87.41 -24.44 -21.48    -21.48 -21.48         77.78   
flags                   74.87 -12.31 -13.33     -9.74 -12.82         72.31   
diggle                 100.00   0.00   0.00      0.00   0.00        100.00   
heart                   84.26  -0.33  -0.98     -0.33   0.00         86.23   
sleuth                  71.11   1.11   1.11      1.11   0.00         76.67   
apnea2                  92.00  -0.21   0.00      0.00  -0.84         90.11   
auto-mpg                97.72   0.51 -14.68    -14.68 -16.96         94.18   
churn                   79.30  -0.43  -0.23     -0.23  -0.26         78.77   
nyc                     60.29   0.00   0.00      0.31   0.31         66.94   
zoo                     71.11  -8.89   0.00     -2.22   0.00         77.78   

                                          
model     pandas   tfdv autogluon     rf  
dataset                                   
cancer      0.00   0.00      0.00   0.00  
mfeat       0.00   0.00      0.00   0.00  
nursery    -0.36   0.00      0.00   0.00  
audiology  -0.87   0.00     -1.30   0.00  
hayes      -7.41  -7.41     -7.41 -14.07  
supreme     0.00   0.10      0.00   0.15  
flare      -0.62   0.92      0.31  -1.54  
kropt      -6.15  -3.41     -3.41 -12.39  
boxing    -14.81 -11.11    -11.11 -11.11  
flags       4.62   2.05      1.03   0.51  
diggle      0.00   0.00      0.00   0.00  
heart      -1.31  -2.95     -1.31   0.00  
sleuth      0.00   0.00      0.00   0.00  
apnea2     -1.47  -0.84     -0.84   1.05  
auto-mpg    1.27 -17.97    -17.97 -17.97  
churn      -0.85  -0.87     -0.92  -0.92  
nyc         0.00   2.02      2.31   2.31  
zoo       -13.33  -8.89     -6.67  -6.67

In [92]:
regression_output = pd.read_csv("results/results_reg.csv")
name_order = regression_output["name"].unique()
col_order = []
for model in ["linear regression", "random forest"]:
    for feature_type in ["truth", "pandas", "tfdv", "autogluon", "rf"]:
        col_order.append((model, feature_type))
results_reg = regression_output[["name", "model", "linear_holdout", "rf_holdout"]
                               ].rename({"name": "dataset", "linear_holdout":"linear regression", "rf_holdout":"random forest"}, axis=1
                                  ).pivot(index="dataset", columns="model"
                                  ).loc[name_order, col_order]
regression_diffs = lambda x: row_diffs(x, classification=False)
regression_table = (results_reg.apply(regression_diffs, axis=1) * 100).round(2)
regression_table

linear regression                                      random forest  \
model                truth  pandas      tfdv autogluon       rf         truth   
dataset                                                                         
mba                  36.26    4.63      4.63      4.63     4.63         38.42   
vineyard            325.51  172.70    172.70    172.70   -28.88        273.44   
apnea            222453.70  -87.75  -1829.47  -1829.47 -1077.66     137622.06   
accident          43539.13    0.00  41515.82      0.00     0.00      59855.11   

                                                 
model      pandas      tfdv autogluon        rf  
dataset                                          
mba          8.64      8.04      8.64      8.04  
vineyard    40.40     40.40     40.40      3.23  
apnea    -1492.98  -2051.01  -2051.01 -12973.44  
accident     0.00  46594.70      0.00      0.00

In [96]:
print(results_table.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{logistic regression} & \multicolumn{5}{l}{random forest} \\
model &               truth & pandas &   tfdv & autogluon &     rf &         truth & pandas &   tfdv & autogluon &     rf \\
dataset   &                     &        &        &           &        &               &        &        &           &        \\
\midrule
cancer    &               60.00 &   0.00 &   0.00 &      0.00 &   0.00 &         66.67 &   0.00 &   0.00 &      0.00 &   0.00 \\
mfeat     &               96.20 &   0.00 &   0.00 &      0.00 &   0.00 &         94.10 &   0.00 &   0.00 &      0.00 &   0.00 \\
nursery   &               92.06 &   0.02 &   0.00 &      0.00 &   0.00 &         98.24 &  -0.36 &   0.00 &      0.00 &   0.00 \\
audiology &               73.48 &  -1.74 &   0.00 &     -3.48 &   0.00 &         72.17 &  -0.87 &   0.00 &     -1.30 &   0.00 \\
hayes     &               75.56 & -17.04 & -17.04 &    -17.04 & -23.70 &         78.52 &  -7.41 &  

In [97]:
print(regression_table.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{linear regression} & \multicolumn{5}{l}{random forest} \\
model &             truth &  pandas &      tfdv & autogluon &       rf &         truth &   pandas &      tfdv & autogluon &        rf \\
dataset  &                   &         &           &           &          &               &          &           &           &           \\
\midrule
mba      &             36.26 &    4.63 &      4.63 &      4.63 &     4.63 &         38.42 &     8.64 &      8.04 &      8.64 &      8.04 \\
vineyard &            325.51 &  172.70 &    172.70 &    172.70 &   -28.88 &        273.44 &    40.40 &     40.40 &     40.40 &      3.23 \\
apnea    &         222453.70 &  -87.75 &  -1829.47 &  -1829.47 & -1077.66 &     137622.06 & -1492.98 &  -2051.01 &  -2051.01 & -12973.44 \\
accident &          43539.13 &    0.00 &  41515.82 &      0.00 &     0.00 &      59855.11 &     0.00 &  46594.70 &      0.00 &      0.00 \\
\bottomrule
\end{tabular}



In [94]:
df = pd.read_csv("datasets/articles.csv", encoding="ISO-8859-1")
df

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business
...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business


In [95]:
table_cols = results_table.columns.drop([("logistic regression", "truth"), ("random forest", "truth")])
performance = pd.DataFrame(columns=table_cols)

#underperforms
for model in ['logistic regression', 'random forest']:
    for label in ["pandas", "tfdv", "autogluon", "rf"]:
        if model == "logistic regression":
            reg_model = "linear regression"
        else:
            reg_model = "random forest"
        performance.loc["Underperforms truth", (model, label)] = (results_table[(model, label)] < 0).sum() + (regression_table[(reg_model, label)] < 0).sum()
        performance.loc["Matches truth", (model, label)] = (results_table[(model, label)] == 0).sum() + (regression_table[(reg_model, label)] == 0).sum()
        performance.loc["Outperforms truth", (model, label)] = (results_table[(model, label)] > 0).sum() + (regression_table[(reg_model, label)] > 0).sum()
performance

logistic regression                    random forest       \
model                            pandas tfdv autogluon  rf        pandas tfdv   
Underperforms truth                  11    9        11  10            11    9   
Matches truth                         6    9         7  10             7    6   
Outperforms truth                     5    4         4   2             4    7   

                                  
model               autogluon rf  
Underperforms truth        10  8  
Matches truth               7  8  
Outperforms truth           5  6

In [98]:
print(performance.to_latex())

\begin{tabular}{lllllllll}
\toprule
{} & \multicolumn{4}{l}{logistic regression} & \multicolumn{4}{l}{random forest} \\
model &              pandas & tfdv & autogluon &  rf &        pandas & tfdv & autogluon & rf \\
\midrule
Underperforms truth &                  11 &    9 &        11 &  10 &            11 &    9 &        10 &  8 \\
Matches truth       &                   6 &    9 &         7 &  10 &             7 &    6 &         7 &  8 \\
Outperforms truth   &                   5 &    4 &         4 &   2 &             4 &    7 &         5 &  6 \\
\bottomrule
\end{tabular}

